In [101]:
# generate sum pairs
from random import seed
from random import randint
seed(1234)
from numpy import array, argmax
from pprint import pprint
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.metrics import Accuracy
# from keras.

In [45]:
def generate_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_patterns = [randint(1, largest) for _  in range(n_numbers)]
#         print(in_patterns)
        out_pattern = sum(in_patterns)
        X.append(in_patterns)
        y.append(out_pattern)
    return X, y

In [46]:
n_samples = 1
n_numbers = 2
largest = 10


In [47]:
X, y = generate_sum_pairs(n_samples, n_numbers, largest)

In [48]:
print(X, y)

[[8, 2]] [10]


In [49]:
#get the maximum length of the sequence 


In [50]:
import numpy as np
import random
random.seed(1234)
# import sklearn.preprocessing.MinMaxScaler

In [51]:
# this is the generating stage , this function only egenrate one number
c = randint(0, 99)

In [52]:
def generate_sequence(length, n_features):
    return [randint(0, n_features-1) for _ in range(length)]

In [60]:
eg = generate_sequence(25, 100)

In [61]:
# rescale it into a format that suitable for LSTM 
def one_hot_encode(sequence, n_features):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_features)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)

In [67]:
encoded = one_hot_encode(eg, 100)
encoded

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [63]:
#how to decode
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [64]:
decoded = one_hot_decode(encoded)

In [72]:
pprint(decoded)

[4,
 85,
 88,
 10,
 12,
 98,
 45,
 30,
 2,
 3,
 2,
 44,
 82,
 79,
 61,
 78,
 59,
 19,
 11,
 23,
 91,
 14,
 1,
 64,
 62]


In [73]:
# reshape one hot encoded sequence 
# because there will be 100 integer, so the feature is 100. 

In [96]:
# wrap it all above into a function
def generate_example(length, n_feature, out_index):
    sequence = generate_sequence(length, n_feature)
    encoded = one_hot_encode(sequence, n_feature)
    X = encoded.reshape((1, length, n_feature))
    y = encoded[out_index].reshape(1, n_feature)  # 25 ge time step 的其中一个
    return X, y
    

In [97]:
X, y = generate_example(25, 100, 2)

In [98]:
X.shape

(1, 25, 100)

In [99]:
y.shape

(1, 100)

In [103]:
# define model
length = 5
n_features = 10
out_index = 2
model = Sequential()
model.add(LSTM(25, input_shape=(length, n_features)))
model.add(Dense(n_features, activation= 'softmax'))
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25)                3600      
_________________________________________________________________
dense (Dense)                (None, 10)                260       
Total params: 3,860
Trainable params: 3,860
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
for i in range(10000):
    X, y = generate_example(length, n_features, out_index)
    model.fit(X, y, epochs=1, verbose=2)

1/1 - 1s - loss: 2.3404 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.3073 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2708 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.3409 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.3470 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.3774 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.4819 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2301 - accuracy: 1.0000
1/1 - 0s - loss: 2.3357 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.4192 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.3823 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2099 - accuracy: 1.0000
1/1 - 0s - loss: 2.3149 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2767 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2330 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.4432 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.1564 - accuracy: 1.0000
1/1 - 0s - loss: 2.2268 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.1854 - accuracy: 1.0000
1/1 - 0s - loss: 2.4357 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2463 - accuracy: 0.0000e+00
1/1 - 0s - loss: 2.2860 - acc

In [106]:
# evaluate the model
correct = 0
for i in range(100):
    X, y = generate_example(length, n_features, out_index)
    yhat = model.predict(X)
    if one_hot_decode(yhat) == one_hot_decode(y):
        correct += 1

print(f'Accuracy {(correct/100)*100.0} %')

Accuracy 99.0 %


In [107]:
# make the model to predict , with the model
# on new data
X, y = generate_example(length, n_features, out_index)
yhat = model.predict(X)
print(f'Sequence is {[one_hot_decode(x) for x in X]}')
print(f'expected y is {one_hot_decode(y)}')
print(f'predicted is {one_hot_decode(yhat)}')


Sequence is [[0, 2, 1, 4, 7]]
expected y is [1]
predicted is [1]
